# Workflow testing


In [1]:
import sys
import os
import json

import numpy as np
import scipy.stats
from matplotlib import pyplot as plt
import matplotlib.lines as mlines

from matplotlib.path import Path
from matplotlib.patches import PathPatch

%matplotlib ipympl

import warnings
warnings.filterwarnings('ignore', module='numpy')
warnings.filterwarnings('ignore')

import importlib
try:
    import bumps
    from bumps import dream
    HAS_BUMPS  = True
except:
    print("No bumps")
    HAS_BUMPS = False

if HAS_BUMPS:
    from tron.bayesian_analysis import fit_uncertainties
    importlib.reload(fit_uncertainties)

from tron.bayesian_analysis import summary_plots

In [2]:
# Data analysis directory
project_dir = os.path.expanduser('~/git/TrON/example_analysis')

# Upper-level data directory for the time-resolved data
data_dir = os.path.join(project_dir, 'data')

# Directory where we store dynamic fit results
dyn_model_dir = os.path.join(project_dir, 'dyn-fitting')



# Auto-reduction directory
#ar_dir = '/SNS/REF_L/IPTS-30384/shared/autoreduce/'

# Directory with fit results
#fit_dir = '/SNS/users/m2d/git/nh3-analysis/july2023/dyn-fitting/'

dynamic_run = 207168 # Cycle 1

#dyn_data_dir = '/SNS/REF_L/IPTS-30384/shared/dynamic/Cu-F/30s'
dyn_fit_dir = os.path.join(dyn_model_dir, '207168-dyn/results-30s-bck')



In [4]:
from tron.bayesian_analysis import template, fitting_loop
importlib.reload(template)
importlib.reload(fitting_loop)

init_json = os.path.join(project_dir, 'dyn-fitting', '207161', '__model-expt.json') 
final_json = os.path.join(project_dir, 'dyn-fitting', '207169', '__model-expt.json')

template_str = template.create_model(init_json, final_json,True)

with open('model-loop-tmp.py', 'w') as fd:
    fd.write(template_str)

results_dir = os.path.join(dyn_model_dir, '207168-dyn/results-30s-bck-test')
fitting_loop.execute_fit(dynamic_run, data_dir, 'model-loop-tmp.py', init_json, final_json, results_dir)


Model: model-loop-tmp
Data: /Users/m2d/git/TrON/example_analysis/data
Results: /Users/m2d/git/TrON/example_analysis/dyn-fitting/207168-dyn/results-30s-bck-test
Initial state: /Users/m2d/git/TrON/example_analysis/dyn-fitting/207161/__model-expt.json
Final state: /Users/m2d/git/TrON/example_analysis/dyn-fitting/207169/__model-expt.json
                   Initial 	            Step
   Cu interface      11 +- 0.17    	      11 +- 0.17   
         Cu rho     6.5 +- 0.016   	     6.5 +- 0.016  
   Cu thickness     561 +- 0.54    	     561 +- 0.54   
  SEI interface    10.9 +- 6.2     	    10.9 +- 6.2    
       SEI irho 0.000705 +- 0.0063  	 0.000705 +- 0.0063 
        SEI rho     3.8 +- 0.094   	     3.8 +- 0.094  
  SEI thickness     187 +- 5.1     	     187 +- 5.1    
  THF interface    56.1 +- 1.7     	    56.1 +- 1.7    
        THF rho    6.25 +- 0.034   	    6.25 +- 0.034  
   Ti interface    1.92 +- 0.59    	    1.92 +- 0.59   
         Ti rho  -0.583 +- 0.03    	  -0.583 +- 0.03   
 